## Курсовая работа

В рамках курсовой работы на практике изучить работу многослойного перцептрона и применить его для решения задачи. Оценить параметры какой-либо песни и предсказать её популярность
Для реализации задачи использован язык программирования Python и 
Spotify API.


**Шаг 1.** Импортируем библиотеки

In [1]:
import numpy as np
import pandas as pd
import random 
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.optimizers import *
from sklearn.model_selection import train_test_split
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
import requests
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import json

**Шаг 2.**  Импортируем датасет

In [2]:
random.seed(13)

a = pd.read_csv('song_data.csv')
a.drop(columns=["song_name"], inplace=True)
a.sample(3)

,song_popularity,song_duration_ms,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,audio_mode,speechiness,tempo,time_signature,audio_valence
4895,5,190933,0.0609,0.837,0.829,0.246000,10,0.0709,-7.243,0,0.1250,104.338,4,0.722
15173,45,171250,0.0849,0.786,0.843,0.000265,7,0.0856,-5.824,1,0.0401,124.940,4,0.693
7803,55,283493,0.3740,0.488,0.209,0.000005,10,0.1030,-19.549,1,0.0369,120.630,4,0.362


**Шаг 3.** Нормализация данных

In [3]:
def normalize(raw, max_tempo = None, max_duration = None):
    raw["song_popularity"] /= 100
    raw['key'] += 1
    raw['key'] /= 12

    raw["loudness"] *= (-1/60)
    raw["loudness"] = abs(raw["loudness"])

    raw["time_signature"] +=3
    raw["time_signature"] /=8
    if not max_tempo:
        max_tempo = raw["tempo"].max()
    raw["tempo"] = raw["tempo"] / max_tempo

    if not max_duration:
        max_duration = raw['song_duration_ms'].max()
    raw["song_duration_ms"]  /= max_duration
    return raw, max_tempo, max_duration

In [4]:
a , max_tempo, max_duration = normalize(a)
a.sample(3)
a.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
song_popularity,18835.0,0.529919,0.219057,0.000000,0.400000,0.560000,0.690000,1.000000
song_duration_ms,18835.0,0.121273,0.033283,0.006669,0.102448,0.117435,0.134962,1.000000
acousticness,18835.0,0.258539,0.288719,0.000001,0.024100,0.132000,0.424000,0.996000
danceability,18835.0,0.633348,0.156723,0.000000,0.533000,0.645000,0.748000,0.987000
energy,18835.0,0.644995,0.214101,0.001070,0.510000,0.674000,0.815000,0.999000
instrumentalness,18835.0,0.078008,0.221591,0.000000,0.000000,0.000011,0.002570,0.997000
key,18835.0,0.524100,0.301216,0.083333,0.250000,0.500000,0.750000,1.000000
liveness,18835.0,0.179650,0.143984,0.010900,0.092900,0.122000,0.221000,0.986000
loudness,18835.0,0.124132,0.063781,0.000867,0.081800,0.109250,0.150733,0.646133
audio_mode,18835.0,0.628139,0.483314,0.000000,0.000000,1.000000,1.000000,1.000000


**Шаг 4.**  Делим данные

* Обучающая выборка: 70%
* Тестовая выборка: 30%

In [5]:
X = a.iloc[:,1:].values
y = a.iloc[:,1].values

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.3)


# Начинаем построение модели
___

In [30]:
seed = 7
np.random.seed(seed)



model = Sequential()
model.add(Dense(500, input_dim=13, activation= "relu"))
model.add(Dense(200, activation= "relu"))
model.add(Dense(1))


model.compile(loss='mean_squared_error', optimizer=Adam(lr=1e-5), metrics=["mean_absolute_percentage_error"])

history = model.fit(X_train, y_train, epochs=200, batch_size=1000, shuffle=True)


Epoch 1/200
14/14 [==============================] - 0s 4ms/step - loss: 0.0023 - mean_absolute_percentage_error: 33.6483
Epoch 2/200
14/14 [==============================] - 0s 5ms/step - loss: 0.0018 - mean_absolute_percentage_error: 29.3258
Epoch 3/200
14/14 [==============================] - 0s 5ms/step - loss: 0.0015 - mean_absolute_percentage_error: 26.6553
Epoch 4/200
14/14 [==============================] - 0s 5ms/step - loss: 0.0013 - mean_absolute_percentage_error: 24.4491
Epoch 5/200
14/14 [==============================] - 0s 5ms/step - loss: 0.0012 - mean_absolute_percentage_error: 22.8104
Epoch 6/200
14/14 [==============================] - 0s 5ms/step - loss: 0.0011 - mean_absolute_percentage_error: 21.5314
Epoch 7/200
14/14 [==============================] - 0s 5ms/step - loss: 9.6972e-04 - mean_absolute_percentage_error: 20.4145
Epoch 8/200
14/14 [==============================] - 0s 6ms/step - loss: 9.0157e-04 - mean_absolute_percentage_error: 19.5510
Epoch 9/200
14/1

Вычисляем конечные потери

In [31]:
print(f"[INFO] LOSS WHILE TRAINING {model.evaluate(X_train, y_train)}")
print(f"[INFO] LOSS WHILE TESTING {model.evaluate(X_test, y_test)}")

412/412 [==============================] - 0s 804us/step - loss: 9.8460e-06 - mean_absolute_percentage_error: 2.0675
[INFO] LOSS WHILE TRAINING [9.845983186096419e-06, 2.0675017833709717]
177/177 [==============================] - 0s 1ms/step - loss: 2.0956e-05 - mean_absolute_percentage_error: 2.3052
[INFO] LOSS WHILE TESTING [2.095632953569293e-05, 2.305208444595337]


In [11]:

def get_song_data(URL):

    CLIENT_ID = 'a4368e3d33f74bd5be1d9ba220d8e5c5'      # client ID
    CLIENT_SECRET = '6b9de796d08f421b9b669d303c941bd4'  # client secret
    REDIRECT_URI = 'http://localhost:8888/callback/'    # redirected URI
    SCOPE = 'ugc-image-upload user-read-playback-state user-modify-playback-state user-read-currently-playing user-read-private user-read-email user-follow-modify user-follow-read user-library-modify user-library-read streaming app-remote-control user-read-playback-position user-top-read user-read-recently-played playlist-modify-private playlist-read-collaborative playlist-read-private playlist-modify-public'
    USER = 't1tdus0slwz6i5apmykqtjcgx'                  # имя пользователя
    AUTH_URL = 'https://accounts.spotify.com/api/token'
    CACHE = '.spotipyoauthcache'

    sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=SCOPE, client_id=CLIENT_ID, client_secret=CLIENT_SECRET, redirect_uri= REDIRECT_URI, username=USER))

    token =sp.auth_manager.get_access_token()
    token= token['access_token']
    
    ID =URL.split("?")[0].split("/")[-1]

    url = f"https://api.spotify.com/v1/audio-features/{ID}"

    headers = {'Authorization': f'Bearer {token}'}
    r = requests.get(url, headers=headers)
    parsed = json.loads(r.text)
    song_features = pd.json_normalize(parsed)
    song_features_df = pd.concat([song_features])

    song_data = []
    song_data.append(0)
    song_data.append(song_features_df['duration_ms'].values[0])
    song_data.append(song_features_df['acousticness'].values[0])
    song_data.append(song_features_df['danceability'].values[0])
    song_data.append(song_features_df['energy'].values[0])
    song_data.append(song_features_df['instrumentalness'].values[0])
    song_data.append(song_features_df['key'].values[0])
    song_data.append(song_features_df['liveness'].values[0])
    song_data.append(song_features_df['loudness'].values[0])
    song_data.append(song_features_df['mode'].values[0])
    song_data.append(song_features_df['speechiness'].values[0])
    song_data.append(song_features_df['tempo'].values[0])
    song_data.append(song_features_df['time_signature'].values[0])
    song_data.append(song_features_df['valence'].values[0])
    raw = pd.DataFrame([song_data], columns=['song_popularity','song_duration_ms','acousticness','danceability','energy','instrumentalness','key','liveness','loudness','audio_mode','speechiness','tempo','time_signature','audio_valence'])


    return raw


In [21]:
r = get_song_data("https://open.spotify.com/track/04m2nJR2Ia6d1ZOt8gi5xA?si=c3009033d57744d0")
r,max_tempo,max_duration = normalize(r,max_tempo,max_duration)
r = r.to_numpy()
r = np.delete(r, 0)


r=r[np.newaxis,:]

predictions = model.predict(r)
print(f"Предполагаемая популярность данного трека: {round(predictions[0][0] * 100,2)}%")


/var/folders/wm/ssp6fywx7ys05n3_b5ddzc_r0000gn/T/ipykernel_89690/2672411883.py:13: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  token =sp.auth_manager.get_access_token()


Предполагаемая популярность данного трека: 33.71%
